In [95]:
import requests
import json
import pandas as pd
import time
from dateutil.parser import parse
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [174]:
apikey = 'X1o3cmgN7nsBDsInu4km3XtlPD051oPC'
nickname = '포뇨의브루스'
wordType = 'matche'
main_url = 'https://api.neople.co.kr/cy/'
player_url = 'https://api.neople.co.kr/cy/players/'
matches_url = 'https://api.neople.co.kr/cy/matches/'
api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
playerdata = requests.get(api)
playerId = playerdata.json()['rows'][0]['playerId']
train = pd.read_csv('train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# X_train = train
# X_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# y_train = pd.read_csv('y_train.csv')
# y_train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [82]:
# 함수 정리
def get_playerid(nickname):#플레이어 고유 id를 구하는 함수
    wordType = 'matche'
    main_url = 'https://api.neople.co.kr/cy/'
    player_url = 'https://api.neople.co.kr/cy/players/'
    matches_url = 'https://api.neople.co.kr/cy/matches/'
    api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
    playerdata = requests.get(api)
    playerId = playerdata.json()['rows'][0]['playerId']
    return playerId

#매치 id 가져오기
def get_matches(playerId):
    matchids = []
    gameTypeId = 'rating'
    #검색 시작일
    startDate='2022-01-01'
    endDate='2022-03-30'
    limit = 1000
    cc = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate
    playdata = requests.get(cc)
    playdata_info= playdata.json()
    while True:
        try:
            for matche in playdata_info['matches']['rows']:
                matchids.append(matche['matchId'])
            next = playdata_info['matches']['next']
            cc_next = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate+'&next='+next    
            playdata = requests.get(cc_next)
            playdata_info= playdata.json()
            time.sleep(0.5)
        except:
            break
    return matchids


def win_and_lose(matchId):#매치 id를 들고와서 승리팀 조합과,패배팀 조합으로 나눠줌
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3)
    lose_team = []
    win_team = []
    lose_ch = []
    win_ch = []
    try:
        if game.json()['teams'][0]['result'] == 'lose':
            for lose_player in game.json()['teams'][0]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][1]['players']:
                win_team.append(win_player)
        elif game.json()['teams'][0]['result'] == 'win':
            for lose_player in game.json()['teams'][1]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][0]['players']:
                win_team.append(win_player)
        lose_ch =[]
        win_ch = []
        for i in range(0,10):
            if game.json()['players'][i]['playerId'] in lose_team:
                lose_ch.append(game.json()['players'][i]['playInfo']['characterName'])
            elif game.json()['players'][i]['playerId'] in win_team:
                win_ch.append(game.json()['players'][i]['playInfo']['characterName'])
    except:pass
    return win_ch,lose_ch



In [204]:
namelist = ['멍댕이냥냥펀치','달빛아재','스안치아','천지신령부붉은개']

for target in namelist:
    playerId = get_playerid(target)
    matchids = get_matches(playerId)
    win = []
    lose = []
    for matchid in matchids:
        winer_team,loser_team = win_and_lose(matchid)
        time.sleep(0.2)
        win.append(winer_team)
        lose.append(loser_team)
    test = pd.DataFrame(win,columns=['c1','c2','c3','c4','c5'])
    test['win']=1
    test2 = pd.DataFrame(lose,columns=['c1','c2','c3','c4','c5'])
    test2['win']=0
    test3 = pd.concat([test,test2],join='inner')
    test3 = test3.reset_index(drop=True, inplace=False)

    test3 = test3.replace('로라스',1).replace('휴톤',2).replace('루이스',3).replace('타라',4).replace('트리비아',5).replace('카인',6).replace('레나',7).replace('드렉슬러',8).replace('도일',9).replace('토마스',10).replace('나이오비',11).replace('시바',12).replace('웨슬리',13).replace('스텔라',14).replace('앨리셔',15).replace('클레어',16).replace('다이무스',17).replace('이글',18).replace('마를렌',19).replace('샬럿',20).replace('윌라드',21).replace('레이튼',22).replace('미쉘',23).replace('린',24).replace('빅터',25).replace('카를로스',26).replace('호타루',27).replace('트릭시',28).replace('히카르도',29).replace('까미유',30).replace('자네트',31).replace('피터',32).replace('아이작',33).replace('레베카',34).replace('엘리',35).replace('마틴',36).replace('브루스',37).replace('미아',38).replace('드니스',39).replace('제레온',40).replace('루시',41).replace('티엔',42).replace('하랑',43).replace('제이',44).replace('벨져',45).replace('리첼',46).replace('리사',47).replace('릭',48).replace('제키엘',49).replace('탄야',50).replace('캐럴',51).replace('라이샌더',52).replace('디아나',53).replace('클리브',54).replace('헬레나',55).replace('에바',56).replace('론',57).replace('레오노르',58).replace('시드니',59).replace('테이',60).replace('티모시',61).replace('엘프리데',62).replace('티샤',63).replace('카로슈',64).replace('라이언',65).replace('파수꾼 A',66).replace('에밀리',67).replace('플로리안',68).replace('케니스',69).replace('이사벨',70).replace('헤나투',71).replace('루드빅',72).replace('멜빈',73)
    #라이샌더뒤로 루드빅을 넣어서 하나씩 밀어야함(임시)
    test3 = test3.fillna(0)
    #탈주자는 0으로 처리
    train = pd.read_csv('train.csv')
    train.drop(['Unnamed: 0'], axis = 1, inplace = True)
    train = pd.concat([train,test3],join='inner')
    train = train.reset_index(drop=True, inplace=False)
    train.to_csv('train.csv')

In [214]:
playerId = get_playerid('똥밟았네')
matchids = get_matches(playerId)
win = []
lose = []
for matchid in matchids:
    winer_team,loser_team = win_and_lose(matchid)
    time.sleep(0.2)
    win.append(winer_team)
    lose.append(loser_team)
test = pd.DataFrame(win,columns=['c1','c2','c3','c4','c5'])
test['win']=1
test2 = pd.DataFrame(lose,columns=['c1','c2','c3','c4','c5'])
test2['win']=0
test3 = pd.concat([test,test2],join='inner')
test3 = test3.reset_index(drop=True, inplace=False)

test3 = test3.replace('로라스',1).replace('휴톤',2).replace('루이스',3).replace('타라',4).replace('트리비아',5).replace('카인',6).replace('레나',7).replace('드렉슬러',8).replace('도일',9).replace('토마스',10).replace('나이오비',11).replace('시바',12).replace('웨슬리',13).replace('스텔라',14).replace('앨리셔',15).replace('클레어',16).replace('다이무스',17).replace('이글',18).replace('마를렌',19).replace('샬럿',20).replace('윌라드',21).replace('레이튼',22).replace('미쉘',23).replace('린',24).replace('빅터',25).replace('카를로스',26).replace('호타루',27).replace('트릭시',28).replace('히카르도',29).replace('까미유',30).replace('자네트',31).replace('피터',32).replace('아이작',33).replace('레베카',34).replace('엘리',35).replace('마틴',36).replace('브루스',37).replace('미아',38).replace('드니스',39).replace('제레온',40).replace('루시',41).replace('티엔',42).replace('하랑',43).replace('제이',44).replace('벨져',45).replace('리첼',46).replace('리사',47).replace('릭',48).replace('제키엘',49).replace('탄야',50).replace('캐럴',51).replace('라이샌더',52).replace('디아나',53).replace('클리브',54).replace('헬레나',55).replace('에바',56).replace('론',57).replace('레오노르',58).replace('시드니',59).replace('테이',60).replace('티모시',61).replace('엘프리데',62).replace('티샤',63).replace('카로슈',64).replace('라이언',65).replace('파수꾼 A',66).replace('에밀리',67).replace('플로리안',68).replace('케니스',69).replace('이사벨',70).replace('헤나투',71).replace('루드빅',72).replace('멜빈',73)
#라이샌더뒤로 루드빅을 넣어서 하나씩 밀어야함(임시)
test3 = test3.fillna(0)
# train = pd.read_csv('train.csv')
# train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# train = pd.concat([train,test3],join='inner')
# train = train.reset_index(drop=True, inplace=False)
# train.to_csv('train.csv')

In [205]:
train = pd.read_csv('train.csv')

In [206]:
train = pd.concat([train,test3],join='inner')
train = train.reset_index(drop=True, inplace=False)

In [207]:
train

,c1,c2,c3,c4,c5,win
0,9,68,8,11.0,32.0,1
1,40,57,62,13.0,14.0,1
2,57,34,62,29.0,58.0,1
3,33,11,71,43.0,2.0,1
4,38,32,9,57.0,61.0,1
...,...,...,...,...,...,...
4531,63,69,56,43.0,5.0,0
4532,49,18,63,11.0,48.0,0
4533,34,5,17,70.0,46.0,0
4534,36,62,12,46.0,64.0,0


In [208]:
train

,c1,c2,c3,c4,c5,win
0,9,68,8,11.0,32.0,1
1,40,57,62,13.0,14.0,1
2,57,34,62,29.0,58.0,1
3,33,11,71,43.0,2.0,1
4,38,32,9,57.0,61.0,1
...,...,...,...,...,...,...
4531,63,69,56,43.0,5.0,0
4532,49,18,63,11.0,48.0,0
4533,34,5,17,70.0,46.0,0
4534,36,62,12,46.0,64.0,0


In [209]:
X_train = train.drop('win',axis=1)
y_train = train.win

In [210]:
X_test = test3.drop('win',axis=1)
y_test = test3.win

In [211]:
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(max_depth = 10,min_child_samples=60,subsample=0.8)
lgb_clf.fit(X_train, y_train)
y_pred = lgb_clf.predict(X_test)

In [212]:
accuracy_score(y_test,y_pred)

0.911504424778761

In [213]:
X_train = pd.concat([X_train,X_test],join='inner')
X_train = X_train.reset_index(drop=True, inplace=False)
y_train = pd.concat([y_train,y_test],join='inner')
y_train = y_train.reset_index(drop=True, inplace=False)
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')

In [190]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [156]:
y_train

""
0
1
2
3
4
...
933
934
935
936
